In [10]:
import pandas as pd

dataset_path = r"C:\Users\a0494\Desktop\Diploma\data\russian_propaganda_tweets\russian_propaganda_tweets_v2.csv"

df = pd.read_csv(dataset_path)
print(df.head())
dataset_path = r"C:\Users\a0494\Desktop\Diploma\data\russian_propaganda_tweets\western_analysts_tweets.csv"

df1 = pd.read_csv(dataset_path, encoding='latin_1')
print(df1.head())

columns_to_keep = ['reply_to', 'language', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'tweet']
df_prop = df[columns_to_keep]
#print(df_prop.head())

columns_to_keep = ['reply_to', 'language', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'tweet']
df_western = df1[columns_to_keep]
df_western.head()


import pandas as pd

df_prop['dataset'] = 0  # Додаємо стовпець 'dataset' до df_prop і заповнюємо його значеннями 0
df_western['dataset'] = 1  # Додаємо стовпець 'dataset' до df_western і заповнюємо його значеннями 1

df_merged = pd.concat([df_prop, df_western])  # Об'єднуємо два датасети
df_merged = df_merged[df_merged['language'] == 'en']
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
columns_to_keep = ['tweet','dataset']
df_merged_cropped = df_merged[columns_to_keep]
nltk.download('stopwords')

df_merged_cropped['word_count'] = df_merged_cropped['tweet'].apply(lambda x: len(str(x).split()))

# Keep only rows where word count is less than or equal to 30
df_merged_cropped = df_merged_cropped[df_merged_cropped['word_count'] <= 30]

# Drop the 'word_count' column if you no longer need it
df_merged_cropped = df_merged_cropped.drop(columns=['word_count'])

vectorizer = CountVectorizer(stop_words = stopwords.words('english'))


# Виконуємо побудову мішка слів
bag_of_words = vectorizer.fit_transform(df_merged_cropped['tweet'])

# Отримуємо список унікальних слів
feature_names = vectorizer.get_feature_names_out()


import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
df_merged_cropped = df_merged_cropped.sample(frac=0.05, random_state=42)
# Ознаки (текстова змінна)
corpus = df_merged_cropped['tweet']

# Цільова змінна
y = df_merged_cropped['dataset']

# Ініціалізація та побудова мішка слів
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# Розділення на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(df_merged_cropped.shape)

          id  conversation_id                             created_at  \
0  1500000.0     1.500000e+18  2022-02-24 14:01:58 GTB Standard Time   
1  1500000.0     1.500000e+18  2022-02-24 14:24:32 GTB Standard Time   
2  1500000.0     1.500000e+18  2022-02-24 14:42:53 GTB Standard Time   
3  1500000.0     1.500000e+18  2022-02-24 14:50:06 GTB Standard Time   
4  1500000.0     1.500000e+18  2022-02-24 14:56:54 GTB Standard Time   

         dat      time  timezone      user_id        username  \
0  2/24/2022  14:01:58       300  344563802.0        tinkzorg   
1  2/24/2022  14:24:32       300  255471924.0      mfa_russia   
2  2/24/2022  14:42:53       300  255471924.0      mfa_russia   
3  2/24/2022  14:50:06       300  114718372.0  russianembassy   
4  2/24/2022  14:56:54       300  255471924.0      mfa_russia   

                            name  place  ... geo source user_rt_id user_rt  \
0  Morgenthau Plan Respecter 40K    NaN  ... NaN    NaN        NaN     NaN   
1                  M

C:\Users\a0494\AppData\Local\Temp\ipykernel_3432\1130214395.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prop['dataset'] = 0  # Додаємо стовпець 'dataset' до df_prop і заповнюємо його значеннями 0
C:\Users\a0494\AppData\Local\Temp\ipykernel_3432\1130214395.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_western['dataset'] = 1  # Додаємо стовпець 'dataset' до df_western і заповнюємо його значеннями 1
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\a0494\AppData\Roamin

(1141, 2)


In [13]:
emb_dim = 128
model = Sequential([Embedding(input_dim=len(vectorizer.get_feature_names_out()), output_dim=emb_dim, input_length=X_train.shape[1]),
                    LSTM(128),
                    Dense(64, activation='relu'),
                    Dense(1, activation='sigmoid')])
print("We r here")
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=5, batch_size=32, validation_data=(X_test.toarray(), y_test))

Epoch 1/10
29/29 [==============================] - 252s 9s/step - loss: 0.6730 - accuracy: 0.6250 - val_loss: 0.6466 - val_accuracy: 0.6638
Epoch 2/10
10/29 [=========>....................] - ETA: 2:42 - loss: 0.6644 - accuracy: 0.6375

KeyboardInterrupt: 